In [1]:
import scanpy as sc
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import anndata2ri
import logging
from scipy.sparse import issparse
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
import anndata as adt
import celltypist
from celltypist import models


rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

/tmp/ipykernel_4809/2551683747.py:18: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [2]:
import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [ ]:
path_dir_folder_processed_data = 'C:/Users/test/Documents/yohan/Data processing pipeline/scRNA/Output/processed_data'

In [ ]:
adatas = [sc.read_h5ad(path_dir_folder_processed_data + '/' + x) for x in os.listdir(path_dir_folder_processed_data)]

In [ ]:
adata = adt.concat(adatas, axis=0, join="outer", index_unique="-")

In [ ]:
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
# log1p transform
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
#could be using Pearson too

In [ ]:
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)
adata.layers["analytic_pearson_residuals"] = csr_matrix(analytic_pearson["X"])

In [ ]:
#Feature selection

In [6]:
%%R
library(scry)

In [7]:
ro.globalenv["adata"] = adata

NameError: name 'adata' is not defined

In [ ]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T

In [ ]:
idx = binomial_deviance.argsort()[-4000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [ ]:
sc.pp.highly_variable_genes(adata, layer="scran_normalization")

In [ ]:
ax = sns.scatterplot(
    data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5
)
ax.set_xlim(None, 1.5)
ax.set_ylim(None, 3)
plt.show()

In [ ]:
#Dimensionality reduction

In [ ]:
adata.X = adata.layers["log1p_norm"]

In [ ]:
#PCA
adata.var["highly_variable"] = adata.var["highly_deviant"]
sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True)

In [ ]:
sc.pl.pca_scatter(adata, color="total_counts")

In [ ]:
#t-SNE
sc.tl.tsne(adata, use_rep="X_pca")

In [ ]:
sc.pl.tsne(adata, color="total_counts")

In [ ]:
#UMAPS
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="total_counts")

In [ ]:
#Genes high Var
sc.pl.pca_variance_ratio(adata, log=True) # put it at the right place, see the most variable genes

In [ ]:
#Mapping
sc.pp.neighbors(adata, n_pcs=30) #adapter le nombre selon valeur coude trouve avec PCA
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata)